#### Auto-fill Questionnaire using Chain of Thought or Few-Shot Examples

This notebook showcases the application of few-shot examples in autofilling questionnaires. It utilizes a json file (`cot_examples.json`) to
provide the LLM with example responses for some use-cases.

By leveraging these few-shot examples, we can enable seamless completion of lengthy questionnaires, minimizing manual effort and improving overall efficiency.


In [2]:
from risk_atlas_nexus.blocks.inference import (
    RITSInferenceEngine,
    WMLInferenceEngine,
    OllamaInferenceEngine,
    VLLMInferenceEngine,
)
from risk_atlas_nexus.blocks.inference.params import (
    InferenceEngineCredentials,
    RITSInferenceEngineParams,
    WMLInferenceEngineParams,
    OllamaInferenceEngineParams,
    VLLMInferenceEngineParams,
)

from copy import deepcopy
from risk_atlas_nexus.library import RiskAtlasNexus

/Users/dhaval/Projects/Usage-Governance/risk-atlas-nexus/src/risk_atlas_nexus/toolkit/job_utils.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


##### Risk Atlas Nexus uses Large Language Models (LLMs) to infer risks dimensions. Therefore requires access to LLMs to inference or call the model.

**Available Inference Engines**: WML, Ollama, vLLM, RITS. Please follow the [Inference APIs](https://github.com/IBM/risk-atlas-nexus?tab=readme-ov-file#install-for-inference-apis) guide before going ahead.

_Note:_ RITS is intended solely for internal IBM use and requires TUNNELALL VPN for access.


In [ ]:
inference_engine = OllamaInferenceEngine(
    model_name_or_path="granite3.2:8b",
    credentials=InferenceEngineCredentials(api_url="OLLAMA_API_URL"),
    parameters=OllamaInferenceEngineParams(
        num_predict=1000, temperature=0, repeat_penalty=1, num_ctx=8192
    ),
)

# inference_engine = WMLInferenceEngine(
#     model_name_or_path="ibm/granite-20b-code-instruct",
#     credentials={
#         "api_key": "WML_API_KEY",
#         "api_url": "WML_API_URL",
#         "project_id": "WML_PROJECT_ID",
#     },
#     parameters=WMLInferenceEngineParams(
#         max_new_tokens=1000, decoding_method="greedy", repetition_penalty=1
#     ),
# )

# inference_engine = VLLMInferenceEngine(
#     model_name_or_path="ibm-granite/granite-3.1-8b-instruct",
#     credentials=InferenceEngineCredentials(
#         api_url="VLLM_API_URL", api_key="VLLM_API_KEY"
#     ),
#     parameters=VLLMInferenceEngineParams(max_tokens=1000, temperature=0.7),
# )

# inference_engine = RITSInferenceEngine(
#     model_name_or_path="ibm-granite/granite-3.1-8b-instruct",
#     credentials={
#         "api_key": "RITS_API_KEY",
#         "api_url": "RITS_API_URL",
#     },
#     parameters=RITSInferenceEngineParams(max_tokens=1000, temperature=0.7),
# )

[2025-05-19 18:30:45:593] - INFO - RiskAtlasNexus - OLLAMA inference engine will execute requests on the server at localhost:11434.
[2025-05-19 18:30:45:640] - INFO - RiskAtlasNexus - Created OLLAMA inference engine.


##### Create an instance of RiskAtlasNexus

_Note: (Optional)_ You can specify your own directory in `RiskAtlasNexus(base_dir=<PATH>)` to utilize custom AI ontologies. If left blank, the system will use the provided AI ontologies.


In [4]:
risk_atlas_nexus = RiskAtlasNexus()

[2025-05-19 18:30:45:815] - INFO - RiskAtlasNexus - Created RiskAtlasNexus instance. Base_dir: None


#### Defining Few-Shot Examples for Auto-Assist Functionality

This cell showcases the template used in `risk_atlas_nexus/data/templates/cot_examples.json` to provide few-shot
examples for auto-assist functionality.

**Template Structure:**

- Each question is associated with a list of example intents and
  corresponding answers.
- The format is:
  - `Question`
  - `- intents:` (list of example intents)
  - `- answers:` (list of corresponding answers for the respective intents above)

In this notebook, we're using a simplified template to cover 4 questions
from the Airo questionnaire:

1. System environment
2. Intended domain
3. Utilized techniques
4. Targeted user group

**Customization:**

To adapt this auto-assist functionality to custom questionnaires, users
need to provide their own set of questions, example intents, and
corresponding answers in a json file (e.g., `cot_examples.json`). This will enable
the LLM to learn from these few-shot examples and generate responses for
unseen queries.


In [13]:
import json
import os
from risk_atlas_nexus.data import get_templates_path

with open(os.path.join(get_templates_path(), "cot_examples.json")) as f:
    cot_data = json.load(f)
print(json.dumps(cot_data[-2], indent=4))

{
    "question": "What is the application of the system?",
    "examples": [
        {
            "intent": "Provide a dashboard to track sales performance by region and product line",
            "answer": "Natural Language Processing (NLP): Analyze sales data to identify trends, patterns, and correlations between regions, products, and sales metrics.\n Machine Learning (ML): Develop predictive models to forecast future sales performance based on historical data and seasonal fluctuations.\n Computer Vision: Integrate with CRM systems to visualize sales pipelines, track customer interactions, and monitor pipeline health.",
            "explanation": null
        },
        {
            "intent": "Automate the process of onboarding new employees, including generating necessary documents and assigning benefits",
            "answer": "Robotic Process Automation (RPA): Automate manual tasks such as generating new hire paperwork, updating HR systems, and scheduling interviews. \n Docume

There are two ways to use the inference engine to get the LLM outputs. `generate_zero_shot_output` which gives the zero-shot output for the question and `generate_few_shot_output` which gives the output using few-shot examples defined above.


In [8]:
questions = []
for data in deepcopy(cot_data):
    questions.append({"question": data["question"]})

usecase = "Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers."

results = risk_atlas_nexus.generate_zero_shot_output(
    inference_engine, usecase, questions
)

results

Inferring with OLLAMA: 100%|██████████| 7/7 [00:18<00:00,  2.66s/it]


['Customer service/support',
 'The system is used in a digital environment, specifically designed for online interactions between customers and support agents.',
 'The system employs a variety of techniques, including multi-modal approaches such as Document Question/Answering, Image and text to text, Video and text to text, and visual question answering. In the realm of Natural Language Processing, it utilizes methods like feature extraction, fill mask, question answering, sentence similarity, summarization, table question answering, text classification, text generation, token classification, translation, and zero shot classification. For computer vision, it uses image classification, image segmentation, text to image, and object detection. In the audio domain, it applies audio classification, audio to audio, and text to speech. Additionally, it incorporates tabular methods such as tabular classification and tabular regression. Lastly, it leverages reinforcement learning.',
 'The inten

In [6]:
usecase = "Generate personalized, relevant responses, recommendations, and summaries of claims for customers to support agents to enhance their interactions with customers."

risk_atlas_nexus.generate_few_shot_output(usecase, cot_data, inference_engine)

Inferring with OLLAMA: 100%|██████████| 7/7 [00:35<00:00,  5.06s/it]


['Customer service/support',
 'Customer Service or Claims Support Departments',
 'Natural language processing: text generation and summarization',
 'Customer Support Agents',
 'To enhance customer service by providing support agents with personalized, contextually relevant information and recommendations, enabling them to address customer inquiries and claims more effectively and efficiently.',
 'Natural Language Generation (NLG): Develop AI models to generate personalized, context-aware responses and summaries for customer interactions. \nSentiment Analysis: Use NLP to understand customer sentiment and tailor responses accordingly. \nRecommendation Engine: Analyze customer data to suggest relevant products, services, or solutions based on individual needs and preferences. \nChatbot Integration: Integrate with customer support platforms to provide real-time, automated assistance to support agents.',
 'Claims and Customers']